In [186]:
import pandas as pd
from datetime import datetime
import re

In [187]:
def open_file(filename):
    try:
        f = open(filename, "r")
        text = f.read()
    except:
        f = open(filename, "r",encoding='latin-1')
        text = f.read()
        # encoding='latin-1'
    lines = text.split('\n')
    return lines


def clean_name(name: str) -> str:
    char_add_ons = ['V.O.', "'S COM VOICE",
                    'Cont\'d', 'O.S.', "'S", "VOICE", "INTERCOM"]
    for txt in char_add_ons:
        if txt in name:
            name = name.replace(txt, '')
    return re.sub("[\(\[].*?[\)\]]", "", name.strip())


def get_title(lines):
    series_ind = 0
    for i in range(len(lines)):
        if "STAR TREK: THE NEXT GENERATION" in lines[i] or 'STAR TREK: DEEP SPACE NINE' in lines[i]:
            series_ind = i
            break
    return str(lines[series_ind+2]).strip().replace('"','')


def get_date(lines):
    draft_ind = 0
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    try:
        date = str(lines[series_ind+2])
        date = date.strip()
        print(date)
        final = datetime.strptime(date, '%B %d, %Y')
    except:
        print(lines[series_ind+2])
        final = "AHHH"
    return final



In [188]:
def get_quotes(lines):
#    characters = [x for x in lines if '\t\t\t\t\t' in x]
    char_index = [i for i in range(len(lines)) if '\t\t\t\t\t' in lines[i]]
    scenes_index = [j for j in range(len(lines)) if 'INT.' in lines[j] or 'EXT.' in lines[j]]

    data = []

    for i in char_index:
        name = lines[i].replace('\t\t\t\t\t', '')
        if "FADE OUT" in name:
            continue

        j = i+1
        next_line = lines[j]

        quote = ""

        while next_line != '':
            next_line = next_line.replace('\t', ' ')
            quote += re.sub("[\(\[].*?[\)\]]", "", next_line)
            j = j+1
            next_line = lines[j]

        if quote!="":
            print(quote,lines[j],lines[j-1])
            print(j)
            print(scenes_index)
            scene = lines[max([x for x in scenes_index if x < i])]

            data.append([name, quote,scene])

    test_df = pd.DataFrame(data, columns=['character', 'quote','scene'])
    return (test_df)

def clean_location(loc:str):
    views = ['INT.','EXT.']
    output = loc
    for v in views:
        if v in output:
            output = output[output.index(v)+5:]
    if '(' in output:
        output = output[:output.index('(')-1]
    elif '-' in output:
        output = output[:output.index('-')-1]
    return(output)

def get_view(loc:str):
    output = None
    views = ['INT.','EXT.']
    for v in views:
        if v in loc:
            output=v
    return(output)


def create_df(filepath: str) -> pd.DataFrame:
    lines = open_file(filepath)
    test_df = get_quotes(lines)
    test_df['location']=test_df['scene'].apply(clean_location)
    test_df['view']=test_df['scene'].apply(get_view)
    test_df['character'] = test_df['character'].apply(clean_name)
    test_df['episode'] = get_title(lines)
    test_df['date']=get_date(lines)
    return test_df


In [217]:
full_data=pd.read_csv('complete_data.csv',index_col=0)
date_issues = full_data[ full_data.date=='AHHH'].file.unique()
print(date_issues)

['469.txt' '270.txt' '267.txt' '209.txt' '232.txt' '193.txt' '192.txt'
 '227.txt' '233.txt' '243.txt' '268.txt' '241.txt' '251.txt' '132.txt']


In [276]:
def get_date(lines):
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    date = str(lines[series_ind+2])
    date = date.strip()
    final=date
    spaces = len(date.split())
    continue_b = False
    if ''==date:
        continue_b=True
    elif ',' not in date:
        final = datetime.strptime(date, '%B %d %Y')
    elif spaces ==3:
        final = datetime.strptime(date, '%B %d, %Y')
    elif spaces==2:
        final = datetime.strptime(date, '%B %d,%Y')
    else:
        print('Unknown date format')

    if continue_b:
        pattern = re.compile(r'^STAR TREK:.* CAST*')
        filtered_strings = [s for s in lines if pattern.match(s.strip())]
        date_info = filtered_strings[0]
        date_info=re.search(r'\d{2}/\d{2}/\d{2}', date_info).group()
        final = datetime.strptime(date_info, '%m/%d/%y')
    return final

lines = open_file('scripts_tng/241.txt')
get_date(lines)



date  :  !!!
['            STAR TREK: "Tapestry - 11/30/92 - CAST ']


datetime.datetime(1992, 11, 30, 0, 0)

In [226]:
print('STAR TREK:*CAST')

STAR TREK:*CAST


In [221]:
t='''STAR TREK: "Lower Decks" - 11/17/93 - CAST '''
re.search(r'\d{2}/\d{2}/\d{2}', t).group()

'11/17/93'